In [7]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [9]:
def data():
    # Split train & validation set
    BBB_Train_Decile10 = pd.read_pickle("BBB_Train_Decile10.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile10.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile10, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 20s 6ms/step - loss: 0.2978 - acc: 0.8915 - val_loss: 0.2996 - val_acc: 0.8938
Epoch 35/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.3049 - acc: 0.8934 - val_loss: 0.2966 - val_acc: 0.8925
Epoch 36/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2918 - acc: 0.8941 - val_loss: 0.2979 - val_acc: 0.8912
Epoch 37/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2952 - acc: 0.8967 - val_loss: 0.2994 - val_acc: 0.8938
Epoch 38/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.3054 - acc: 0.8892 - val_loss: 0.2985 - val_acc: 0.8951
Epoch 39/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2960 - acc: 0.8931 - val_loss: 0.2991 - val_acc: 0.8925
Epoch 40/75
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2941 - acc: 0.8947 - val_loss: 0.2986 - val_acc: 0.8925

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1e

3087/3087 [==============================] - 20s 6ms/step - loss: 0.3054 - acc: 0.8834 - val_loss: 0.2899 - val_acc: 0.8938
Epoch 19/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2993 - acc: 0.8892 - val_loss: 0.2773 - val_acc: 0.8977
Epoch 20/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2995 - acc: 0.8847 - val_loss: 0.2897 - val_acc: 0.8873
Epoch 21/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2994 - acc: 0.8886 - val_loss: 0.2913 - val_acc: 0.8977
Epoch 22/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2936 - acc: 0.8876 - val_loss: 0.2793 - val_acc: 0.8990
Epoch 23/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.3022 - acc: 0.8860 - val_loss: 0.2718 - val_acc: 0.9003
Epoch 24/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2954 - acc: 0.8844 - val_loss: 0.2756 - val_acc: 0.8977
Epoch 25/100
3087/3087 [==============================

3087/3087 [==============================] - 20s 6ms/step - loss: 0.2874 - acc: 0.8944 - val_loss: 0.2759 - val_acc: 0.8951
Epoch 77/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2918 - acc: 0.8889 - val_loss: 0.2758 - val_acc: 0.8951
Epoch 78/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2785 - acc: 0.8954 - val_loss: 0.2759 - val_acc: 0.8951
Epoch 79/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2805 - acc: 0.8983 - val_loss: 0.2758 - val_acc: 0.8951
Epoch 80/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2843 - acc: 0.8908 - val_loss: 0.2759 - val_acc: 0.8951
Epoch 81/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2890 - acc: 0.8882 - val_loss: 0.2758 - val_acc: 0.8951
Epoch 82/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2851 - acc: 0.8873 - val_loss: 0.2758 - val_acc: 0.8951
Epoch 83/100
3087/3087 [==============================

Epoch 35/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2926 - acc: 0.8925 - val_loss: 0.2895 - val_acc: 0.9016
Epoch 36/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2895 - acc: 0.8973 - val_loss: 0.2883 - val_acc: 0.9016
Epoch 37/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2896 - acc: 0.8947 - val_loss: 0.2894 - val_acc: 0.8964
Epoch 38/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2916 - acc: 0.8931 - val_loss: 0.2880 - val_acc: 0.8977
Epoch 39/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2890 - acc: 0.8941 - val_loss: 0.2871 - val_acc: 0.8977
Epoch 40/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2849 - acc: 0.8989 - val_loss: 0.2873 - val_acc: 0.9016
Epoch 41/100
3087/3087 [==============================] - 19s 6ms/step - loss: 0.2828 - acc: 0.8970 - val_loss: 0.2882 - val_acc: 0.8977
Epoch 42/100
3087/3087 [=================

...

3087/3087 [==============================] - 11s 4ms/step - loss: 0.3067 - acc: 0.8895 - val_loss: 0.2986 - val_acc: 0.8964
Epoch 100/125
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3053 - acc: 0.8931 - val_loss: 0.2984 - val_acc: 0.8964
Epoch 101/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3109 - acc: 0.8882 - val_loss: 0.2984 - val_acc: 0.8964
Epoch 102/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3033 - acc: 0.8937 - val_loss: 0.2982 - val_acc: 0.8977
Epoch 103/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3134 - acc: 0.8882 - val_loss: 0.2991 - val_acc: 0.8938
Epoch 104/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3191 - acc: 0.8856 - val_loss: 0.2996 - val_acc: 0.8938
Epoch 105/125
3087/3087 [==============================] - 13s 4ms/step - loss: 0.3098 - acc: 0.8869 - val_loss: 0.2988 - val_acc: 0.8951
Epoch 106/125
3087/3087 [=======================

3087/3087 [==============================] - 8s 3ms/step - loss: 0.3371 - acc: 0.8704 - val_loss: 0.3036 - val_acc: 0.8873
Epoch 34/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3212 - acc: 0.8844 - val_loss: 0.3001 - val_acc: 0.8925
Epoch 35/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3273 - acc: 0.8805 - val_loss: 0.2973 - val_acc: 0.8925
Epoch 36/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3259 - acc: 0.8743 - val_loss: 0.2991 - val_acc: 0.8912
Epoch 37/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3216 - acc: 0.8818 - val_loss: 0.2943 - val_acc: 0.8964
Epoch 38/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3257 - acc: 0.8837 - val_loss: 0.2970 - val_acc: 0.8925
Epoch 39/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3245 - acc: 0.8759 - val_loss: 0.2932 - val_acc: 0.8951
Epoch 40/75
3087/3087 [==============================] - 8s 3ms/ste

3087/3087 [==============================] - 7s 2ms/step - loss: 0.3097 - acc: 0.8837 - val_loss: 0.2909 - val_acc: 0.8938
Epoch 18/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2985 - acc: 0.8847 - val_loss: 0.2809 - val_acc: 0.8977
Epoch 19/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2940 - acc: 0.8831 - val_loss: 0.2829 - val_acc: 0.8964
Epoch 20/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3041 - acc: 0.8827 - val_loss: 0.2944 - val_acc: 0.8821
Epoch 21/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2978 - acc: 0.8876 - val_loss: 0.2878 - val_acc: 0.8977
Epoch 22/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2971 - acc: 0.8892 - val_loss: 0.2877 - val_acc: 0.8925
Epoch 23/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3111 - acc: 0.8886 - val_loss: 0.2815 - val_acc: 0.8990

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0

3087/3087 [==============================] - 7s 2ms/step - loss: 0.2882 - acc: 0.8899 - val_loss: 0.2812 - val_acc: 0.9016
Epoch 76/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2935 - acc: 0.8866 - val_loss: 0.2812 - val_acc: 0.8990
Epoch 77/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2899 - acc: 0.8869 - val_loss: 0.2812 - val_acc: 0.8990
Epoch 78/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2870 - acc: 0.8879 - val_loss: 0.2812 - val_acc: 0.8990
Epoch 79/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2873 - acc: 0.8860 - val_loss: 0.2812 - val_acc: 0.8990
Epoch 80/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.2973 - acc: 0.8863 - val_loss: 0.2812 - val_acc: 0.8990
Epoch 81/125
3087/3087 [==============================] - 8s 2ms/step - loss: 0.2888 - acc: 0.8895 - val_loss: 0.2812 - val_acc: 0.9003
Epoch 82/125
3087/3087 [==============================] - 8s 

Epoch 10/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4014 - acc: 0.8280 - val_loss: 0.3679 - val_acc: 0.8601
Epoch 11/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3927 - acc: 0.8432 - val_loss: 0.3521 - val_acc: 0.8666
Epoch 12/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3702 - acc: 0.8507 - val_loss: 0.3355 - val_acc: 0.8769
Epoch 13/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3512 - acc: 0.8656 - val_loss: 0.3381 - val_acc: 0.8536
Epoch 14/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3397 - acc: 0.8626 - val_loss: 0.3181 - val_acc: 0.8782
Epoch 15/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3267 - acc: 0.8776 - val_loss: 0.3026 - val_acc: 0.8951
Epoch 16/50
3087/3087 [==============================] - 8s 2ms/step - loss: 0.3323 - acc: 0.8727 - val_loss: 0.3068 - val_acc: 0.8860
Epoch 17/50
3087/3087 [==============================] 

In [ ]:
0.8297